In [18]:
import pandas as pd
import numpy as np
import pickle

In [42]:
def load_data(day, time):
    # load data
    A_traffics = np.load(f"processed_data/accident_tables/A_traffics_wd_early.npy")
    print(A_traffics.shape)
    print(f"Loading accident and congestion data for {day}, {time}")
    accident_df = pd.read_csv(
        f"processed_data/accident_tables/{day}_{time}_accidents.csv"
    )
    congestion_df = pd.read_csv(
        f"processed_data/congestion_tables/{day}_{time}_congestion.csv"
    )

    # process data
    print(f"Processing data")
    accident_df["date"] = pd.to_datetime(accident_df["date"])
    congestion_df["date"] = pd.to_datetime(congestion_df["date"])

    # Step 1: base mask
    mask_year = (accident_df["date"].dt.year == 2023) & \
                (congestion_df["date"].dt.year == 2023)

    # Step 2: temporary dfs
    acc_tmp = accident_df[mask_year].drop("date", axis=1)
    cong_tmp = congestion_df[mask_year].drop("date", axis=1)

    # Step 3: validity mask
    mask_valid = ~(
        acc_tmp.isna().any(axis=1)
        | cong_tmp.isna().any(axis=1)
        | (acc_tmp == 0).all(axis=1)
    )

    # Step 4: get indices of scenarios to keep
    scenario_indices = acc_tmp.index[mask_valid].tolist()

    # Now filter final dfs + A_traffics
    accident_df_filtered = acc_tmp.loc[mask_valid]
    congestion_df_filtered = cong_tmp.loc[mask_valid]
    A_traffics_filtered = [A_traffics[i] for i in scenario_indices]

    # process accidents
    accident_np = accident_df_filtered.to_numpy()
    accident_counts = accident_np.sum(axis=1)
    P = accident_np / accident_counts[:, np.newaxis]

    # process accidents
    accident_np = accident_df_filtered.to_numpy()
    accident_counts = accident_np.sum(axis=1)
    P = accident_np / accident_counts[:, np.newaxis]

    return P, A_traffics_filtered, congestion_df_filtered, accident_df_filtered

In [43]:
P, A_traffics_filtered, congestion_df_filtered, accident_df_filtered = load_data("wd", "night")

(512, 189, 300)
Loading accident and congestion data for wd, night
Processing data


IndexError: index 513 is out of bounds for axis 0 with size 512

In [23]:
with open('results/model_1_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [ ]:
result = results['wd']['early'][20]

X_opt = result['X']
y_opt = result['y']


In [ ]:
def evaluate_on_test(A_test, P_test, X_opt, y_opt):
    scenario_times = []
    for i in range(len(A_test)):
        # Same objective expression as training
        T_i = ((A_test[i] * X_opt) @ P_test[i, :]).sum()
        scenario_times.append(T_i)
        
    return np.mean(scenario_times)


In [35]:
P_test.shape

(120, 300)

In [30]:
evaluate_on_test(A_traffics[A_indices[0]], P_test, X_opt, y_opt)

IndexError: index 120 is out of bounds for axis 0 with size 120